In [1]:
!pip install whoosh

    100% |████████████████████████████████| 471kB 847kB/s 
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
#function wont work if import statement is inside, must be outside
from whoosh.analysis import Filter
class CustomFilter(Filter):
    # This filter will run for both the index and the query
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

In [1]:
def shopifySearch(df, userQuery):
    #!pip --quiet install whoosh
    from whoosh import index, writing
    from whoosh.fields import Schema, TEXT, ID, STORED
    from whoosh.analysis import RegexTokenizer, LowercaseFilter, StopFilter, StemFilter
    from whoosh import qparser
    from whoosh.qparser import QueryParser, GtLtPlugin, PhrasePlugin, SequencePlugin
    from whoosh import scoring
    import os, os.path # os - portable way of using operating system dependent functionality
    import shutil #High-level file operations
    import pandas
    import nltk
    
    
    #Defining constants for the data paths ***** MODIFY ACCORDINGLY *****
    INDEX_DIR = r"\Desktop"
    
    
    #BUILD SCHEMA ****
    #schema has fields - piece of info for each doc in the index
    customWordFilter = RegexTokenizer()|LowercaseFilter()|CustomFilter(nltk.stem.porter.PorterStemmer().stem)|CustomFilter(nltk.WordNetLemmatizer().lemmatize)

    ixSchema = Schema(comment_ID = ID(stored=True),
                         comment_Subreddit = ID(stored=True),
                         #note analyzer is a wrapper for a tokenizer and zero or more filters -- i.e. allows you to combine them
                         comment_Content = TEXT(analyzer = customWordFilter))
  

    #BUILD INDEX ****
    
    
    if not os.path.exists(INDEX_DIR):
        os.mkdir(INDEX_DIR)


#     if index exists - remove it
#     #Return True if path is an existing directory.
#     if os.path.isdir(INDEX_DIR):
#         #Delete an entire directory tree; path must point to a directory
#         shutil.rmtree(INDEX_DIR)
#     #create the directory for the index
#     os.makedirs(INDEX_DIR)

    #initiate index - takes two inputs, the index directory and the schema for the index
    ix = index.create_in(INDEX_DIR,ixSchema)
    
    
    
    #INDEX COMMENTS ****
    #creating a utility writer 
    #params: index – the whoosh.index.Index to write to.
    #period – the maximum amount of time (in seconds) between commits.
    #limit – the maximum number of documents to buffer before committing/between commits.
    writer = writing.BufferedWriter(ix, period=20, limit=1000)
    try:
        # write each file to index
        # enumerate returns index,value index points too --> index,a[index]
        
        counter1 = 0
        for row in df.iterrows():
            index,data = row
            writer.add_document(comment_ID = data['name'],
                                comment_Subreddit = data['subreddit'],
                                comment_Content = data['body'])
            counter1 = counter1 + 1
            
#             if (counter1 % 100 == 0):
#                 print("already indexed:", counter1+1)

    finally:
        # save the index
        #print("done indexing")
        # *** Note *** -> Must explictly call close() on the writer object to release the write lock and makesure uncommited changes are saved 
        writer.close()   
      
    
    #PARSE USER QUERY ****
    
    #in the query parser --> we pass the DEFAULT field to search and the schema of the index we are searching
    #NOTE: Users can still specify a search on a different field in the schema via --> <fieldname>: <query>
    qp = QueryParser("comment_Content", schema=ix.schema)

     #Once you have a QueryParser object, you can call parse() on it to parse a query string into a query object:
        #default query lang: 
        #If the user doesn’t explicitly specify AND or OR clauses: 
        #by default, the parser treats the words as if they were connected by AND,
        #meaning all the terms must be present for a document to match
        #we will change this 
        #to phrase search "<query>" - use quotes
        
    qp.add_plugin(qparser.GtLtPlugin)   
    #qp.remove_plugin_class(qparser.PhrasePlugin)
    qp.add_plugin(qparser.PhrasePlugin)  
    query = qp.parse(userQuery)
    print("\n\n Query: ")
    print(query)
    print("\n\n")
    
    ##IMPLEMENT SEARCHER ****
    resultsDF = pandas.DataFrame() #creates a new dataframe that's empty to store the results comment content
    with ix.searcher(weighting = scoring.BM25F()) as searcher:
        queryResults = searcher.search(query, limit = None)
        print("Total Number of Results:",len(queryResults))
        print("Number of scored and sorted docs in this Results object:",queryResults.scored_length())
        for result in queryResults:
#             print(result)
#             print("\n",result['comment_ID'])
            resultsDF = resultsDF.append(df.loc[df['name']== result['comment_ID']][['name','subreddit','body']])
            

    #print(dataDf.loc[dataDf['body']==comment].index.values[0])
        
    return resultsDF     



In [3]:
def ShopifySearchEngine(df, userQuery):
    #!pip --quiet install whoosh
    from whoosh import index, writing
    from whoosh.fields import Schema, TEXT, ID, STORED
    from whoosh.analysis import RegexTokenizer, LowercaseFilter, StopFilter, StemFilter
    from whoosh import qparser
    from whoosh.qparser import QueryParser, GtLtPlugin, PhrasePlugin, SequencePlugin
    from whoosh import scoring
    import os, os.path # os - portable way of using operating system dependent functionality
    import shutil #High-level file operations
    import pandas
    import nltk
    
    
    #Defining constants for the data paths ***** MODIFY ACCORDINGLY *****
    INDEX_DIR = r"C:\Users\Jason\MIE490 - Capstone - Shopify\Data\Index2"
    
    
    #BUILD SCHEMA ****
    #schema has fields - piece of info for each doc in the index
    customWordFilter = RegexTokenizer()|LowercaseFilter()|CustomFilter(nltk.stem.porter.PorterStemmer().stem)|CustomFilter(nltk.WordNetLemmatizer().lemmatize)

    ixSchema = Schema(comment_ID = ID(stored=True),
                         comment_Subreddit = ID(stored=True),
                         #note analyzer is a wrapper for a tokenizer and zero or more filters -- i.e. allows you to combine them
                         comment_Content = TEXT(analyzer = customWordFilter))
  

    #BUILD INDEX ****
    
    
    if not os.path.exists(INDEX_DIR):
        os.mkdir(INDEX_DIR)


#     if index exists - remove it
#     #Return True if path is an existing directory.
#     if os.path.isdir(INDEX_DIR):
#         #Delete an entire directory tree; path must point to a directory
#         shutil.rmtree(INDEX_DIR)
#     #create the directory for the index
#     os.makedirs(INDEX_DIR)

    #initiate index - takes two inputs, the index directory and the schema for the index
    ix = index.create_in(INDEX_DIR,ixSchema)
    
    
    
    #INDEX COMMENTS ****
    #creating a utility writer 
    #params: index – the whoosh.index.Index to write to.
    #period – the maximum amount of time (in seconds) between commits.
    #limit – the maximum number of documents to buffer before committing/between commits.
    writer = writing.BufferedWriter(ix, period=20, limit=1000)
    try:
        # write each file to index
        # enumerate returns index,value index points too --> index,a[index]
        
        counter1 = 0
        for row in df.iterrows():
            index,data = row
            writer.add_document(comment_ID = data['name'],
                                comment_Subreddit = data['subreddit'],
                                comment_Content = data['body'])
            counter1 = counter1 + 1
            
#             if (counter1 % 100 == 0):
#                 print("already indexed:", counter1+1)

    finally:
        # save the index
        #print("done indexing")
        # *** Note *** -> Must explictly call close() on the writer object to release the write lock and makesure uncommited changes are saved 
        writer.close()   
      
    
    #PARSE USER QUERY ****
    
    #in the query parser --> we pass the DEFAULT field to search and the schema of the index we are searching
    #NOTE: Users can still specify a search on a different field in the schema via --> <fieldname>: <query>
    qp = QueryParser("comment_Content", schema=ix.schema)

     #Once you have a QueryParser object, you can call parse() on it to parse a query string into a query object:
        #default query lang: 
        #If the user doesn’t explicitly specify AND or OR clauses: 
        #by default, the parser treats the words as if they were connected by AND,
        #meaning all the terms must be present for a document to match
        #we will change this 
        #to phrase search "<query>" - use quotes
        
    qp.add_plugin(qparser.GtLtPlugin)   
    #qp.remove_plugin_class(qparser.PhrasePlugin)
    qp.add_plugin(qparser.PhrasePlugin)  
    query = qp.parse(userQuery)
    print("\n\n Query: ")
    print(query)
    print("\n\n")
    
    ##IMPLEMENT SEARCHER ****
    resultsDF = pandas.DataFrame() #creates a new dataframe that's empty to store the results comment content
    with ix.searcher(weighting = scoring.BM25F()) as searcher:
        queryResults = searcher.search(query, limit = None)
        print("Total Number of Results:",len(queryResults))
        print("Number of scored and sorted docs in this Results object:",queryResults.scored_length())
        for result in queryResults:
#             print(result)
#             print("\n",result['comment_ID'])
            resultsDF = resultsDF.append(df.loc[df['name']== result['comment_ID']][['name','subreddit','body']])
            

    #print(dataDf.loc[dataDf['body']==comment].index.values[0])
        
    return resultsDF     


In [21]:
df = pandas.DataFrame(columns=['text','MI', 'PMI'],index=[0,1,2,3])
df.loc[0] = pandas.Series({'text':"Awesome room service", 'MI':1, 'PMI':0.75})
df.loc[1] = pandas.Series({'text':"Good view", 'MI':0.5, 'PMI':0.75})
df.loc[2] = pandas.Series({'text':"Ok view", 'MI':0, 'PMI':0.5})
df.loc[3] = pandas.Series({'text':"Bad room service", 'MI':-0.5, 'PMI':0})
df.loc[4] = pandas.Series({'text':"Terrible prices", 'MI':-1, 'PMI':-0.75})
df.loc[5] = pandas.Series({'text':"Disastrous prices", 'MI':-1, 'PMI':-1})
df

,text,MI,PMI
0,Awesome room service,1,0.75
1,Good view,0.5,0.75
2,Ok view,0,0.5
3,Bad room service,-0.5,0
4,Terrible prices,-1,-0.75
5,Disastrous prices,-1,-1


In [11]:
###import stuff
import tkinter
from tkinter import *
import tkinter.font as tkFont
import tkinter.ttk as ttk

import os
import datetime
import sqlite3
from tkinter.ttk import Combobox,Treeview,Scrollbar

import pandas

In [12]:
###Create Table
class TableApp(Frame):

    def __init__(self, parent):
        Frame.__init__(self, parent)
        self.CreateTable()
        self.LoadTable()
        self.grid(sticky = (N,S,W,E))
        parent.grid_rowconfigure(0, weight = 1)
        parent.grid_columnconfigure(0, weight = 1)
        
    def CreateTable(self):
        tv = Treeview(self)
        tv['columns'] = ('pointwisemutual', 'mutual', 'viewfull')
        tv.heading("#0", text='Key Phrase')
        tv.column("#0", anchor="w", width=300)
        tv.heading('pointwisemutual', text='PMI')
        tv.column('pointwisemutual', anchor='center', width=50)
        tv.heading('mutual', text='MI')
        tv.column('mutual', anchor='center', width=50)
        tv.heading('viewfull', text='See Full Thread')
        tv.column('viewfull', anchor='center', width=100)
        tv.grid(sticky = (N,S,W,E))
        self.treeview = tv
        self.grid_rowconfigure(0, weight = 1)
        self.grid_columnconfigure(0, weight = 1)
        
    ###Get Table Values
    def LoadTable(self):
        
        MItable = pandas.DataFrame(columns=['text','MI', 'PMI'],index=[0,1,2,3])
        MItable.loc[0] = pandas.Series({'text':"Awesome room service", 'MI':1, 'PMI':0.75})
        MItable.loc[1] = pandas.Series({'text':"Good view", 'MI':0.5, 'PMI':0.75})
        MItable.loc[2] = pandas.Series({'text':"Ok view", 'MI':0, 'PMI':0.5})
        MItable.loc[3] = pandas.Series({'text':"Bad room service", 'MI':-0.5, 'PMI':0})
        MItable.loc[4] = pandas.Series({'text':"Terrible prices", 'MI':-1, 'PMI':-0.75})
        MItable.loc[5] = pandas.Series({'text':"Disastrous prices", 'MI':-1, 'PMI':-1})
        MItable
        
        for (i,row) in MItable.iterrows():
            self.treeview.insert('', 'end', text=row["text"], values=(row["MI"],row["PMI"], 'View'))

In [6]:
###Create Timeline Bar Graph
class TimelineApp(Frame):

    def __init__(self, parent):
        Frame.__init__(self, parent)
        self.CreateGraph()
        self.grid(sticky = (N,S,W,E))
        parent.grid_rowconfigure(0, weight = 1)
        parent.grid_columnconfigure(0, weight = 1)
        
    def CreateGraph(self):
        timetable = pandas.DataFrame(columns=['month','total'],index=[0,1,2])
        timetable.loc[0] = pandas.Series({'month':"Jan", 'total':50})
        timetable.loc[1] = pandas.Series({'month':"Feb", 'total':35})
        timetable.loc[2] = pandas.Series({'month':"Mar", 'total':29})
        timetable.loc[3] = pandas.Series({'month':"Apr", 'total':15})
        timetable.loc[4] = pandas.Series({'month':"May", 'total':10})
        timetable.loc[5] = pandas.Series({'month':"Jun", 'total':47})
        timetable.loc[6] = pandas.Series({'month':"Jul", 'total':38})
        timetable.loc[7] = pandas.Series({'month':"Aug", 'total':7})
        timetable.loc[8] = pandas.Series({'month':"Sep", 'total':23})
        timetable.loc[9] = pandas.Series({'month':"Oct", 'total':19})
        timetable.loc[10] = pandas.Series({'month':"Nov", 'total':28})
        timetable.loc[11] = pandas.Series({'month':"Dec", 'total':33})        
        
        ###month = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
                
        c_width = 55
        c_height = 180
        c = Canvas(self, width=c_width, height=c_height, bg= 'gray')
        c.pack()
        
        ###sizing the graph
        ###y height = max value * y_stretch
        y_height=timetable.total.max()
        y_stretch = 140/y_height
        # gap between lower canvas edge and x axis
        y_gap = 20
        # stretch enough to get all data items in
        x_stretch = 15
        x_width = 28
        # gap between left canvas edge and y axis
        x_gap = 20
        
        c.create_line(0,c_height - y_gap,550,c_height - y_gap)
                
        for i,row in timetable.iterrows():
            # calculate reactangle coordinates (integers) for each bar
            x0 = i * x_stretch + i * x_width + x_gap
            y0 = c_height - (row["total"] * y_stretch + y_gap)
            x1 = i * x_stretch + i * x_width + x_width + x_gap
            y1 = c_height - y_gap
            # draw the bar
            c.create_rectangle(x0, y0, x1, y1, fill="midnightblue")
            # put the y value above each bar
            c.create_text(x1, y0, anchor=SE, text=row["total"])
            c.create_text(x1, c_height-2, anchor=SE, text=row["month"])
            print(i)

In [23]:
###Main Application
class App(tkinter.Tk):

    ###Initialize things
    def __init__(self, parent):
        tkinter.Tk.__init__(self, parent)
        self.parent = parent
        self.initialize()
    
    ###Actually initialize the program
    def initialize(self):
    
        ###Create the main container frames
        firstframe = Frame(self, bg='yellowgreen', width = 1000, height=50, padx=3, pady=3)
        secondframe = Frame(self, bg='gray', width=1000, height=100, padx=3, pady=3)
        thirdframe = Frame(self, bg='white', width = 1000, height = 100, padx=3, pady=3)
        fourthframe = Frame(self, bg='lavender', width = 1000, height = 100, padx=3, pady=3)
        fifthframe = Frame(self, bg='yellowgreen', width = 1000, height = 300, padx=3, pady=3)
        sixthframe = Frame(self, bg='gray', width = 1000, height = 300, padx=3, pady=3)

        ###layout all of the main containers
        firstframe.grid(row=0, sticky="ew")
        secondframe.grid(row=1, sticky="nsew")
        thirdframe.grid(row=3, sticky="ew")
        fourthframe.grid(row=4, sticky="ew")
        fifthframe.grid(row=5, sticky="ew")
        sixthframe.grid(row=6, sticky="ew")
        
        ###first frame = search box
        submit = Button(firstframe, text ="Search", background="yellowgreen", command=self.OnButtonClick)
        
        ###self.entryVariable is the search item
        self.entryVariable = tkinter.StringVar()
        self.entry = tkinter.Entry(firstframe,textvariable=self.entryVariable)
        self.entry.bind("<Return>", self.OnPressEnter)
        self.entryVariable.set("*")
        
        ###top frame layout
        submit.grid(row=0, column=1)
        self.entry.grid(row=0, column=0, sticky='EW')

        ###second frame = summary stats
        statslabel1= Label(secondframe, text="Total Comments", bg="gray")
        statslabel2= Label(secondframe, text="Total Subreddits", bg="gray")
        statslabel3= Label(secondframe, text="Total Positive", bg="gray", fg="darkgreen")
        statslabel4= Label(secondframe, text="Total Neutral", bg="gray", fg="yellow")
        statslabel5= Label(secondframe, text="Total Negative", bg="gray", fg="red")
        
        self.statResult1 = tkinter.StringVar()
        self.statResult2 = tkinter.StringVar()        
        self.statResult3 = tkinter.StringVar()        
        self.statResult4 = tkinter.StringVar()        
        self.statResult5 = tkinter.StringVar()
        
        self.statResult1.set(u"0")
        self.statResult2.set(u"0")
        self.statResult3.set(u"0")
        self.statResult4.set(u"0")
        self.statResult5.set(u"0")
        
        resultlabel1 = Label(secondframe,textvariable=self.statResult1, bg="gray")
        resultlabel2 = Label(secondframe,textvariable=self.statResult2, bg="gray")
        resultlabel3 = Label(secondframe,textvariable=self.statResult3, bg="gray")
        resultlabel4 = Label(secondframe,textvariable=self.statResult4, bg="gray")
        resultlabel5 = Label(secondframe,textvariable=self.statResult5, bg="gray")

        ###second frame layout
        statslabel1.grid(row=0, column=0)
        statslabel2.grid(row=0, column=2)
        statslabel3.grid(row=1, column=0, sticky="w")
        statslabel4.grid(row=1, column=2)
        statslabel5.grid(row=1, column=4)

        resultlabel1.grid(row=0, column=1)
        resultlabel2.grid(row=0, column=3)
        resultlabel3.grid(row=1, column=1)
        resultlabel4.grid(row=1, column=3)
        resultlabel5.grid(row=1, column=5)

        ###third frame = Key Phrase with PMI and MI headers
        keyphraselabel = Label(thirdframe, text="Key Phrases for")
        keyphrasesearch = Label(thirdframe, textvariable=self.entryVariable)
        keyphraseshow = Label(thirdframe, text="Show: ")
        
        choice = StringVar()
        choice.set("all")
        self.RB1 = Radiobutton(thirdframe, text="all", variable=choice, value="all", state="disabled")
        self.RB2 = Radiobutton(thirdframe, text="pos", variable=choice, value="pos", state="disabled")
        self.RB3 = Radiobutton(thirdframe, text="neg", variable=choice, value="neg", state="disabled")
        
        showallcomments = Button(thirdframe, text ="Show All Comments", command=self.SecondWindow)
                
        ###third frame layout
        keyphraselabel.grid(row=0,column=0)
        keyphrasesearch.grid(row=0,column=1)
        keyphraseshow.grid(row=0,column=3)
        self.RB1.grid(row=0, column=4)
        self.RB2.grid(row=0, column=5)
        self.RB3.grid(row=0, column=6)
        showallcomments.grid(row=0, column=7)
        
        ###fourthframe = Table
        TableApp(fourthframe)
        
        ###fifth frame = Timeline Header
        timelinehead = Label(fifthframe, text="Comment Frequency Timeline", bg="yellowgreen")
        
        timelinehead.grid(row=0, column=0)
        
        ###sixth frame = Actual Timeline
        TimelineApp(sixthframe)
        
    ###Click Submit Button and show all search
    def OnButtonClick(self):
        print("OnButtonClick")
        searchTable = shopifySearch(df,self.entryVariable.get())
        print(searchTable)
        self.RB1.configure(state="normal")
        self.RB2.configure(state="normal")
        self.RB3.configure(state="normal")
        self.RB1.select()
        #invoke search function
        
    ###search function to be implemented
        
    ###summary statistics

    ###Click Enter and show all search
    def OnPressEnter(self,event):
        print("OnPressEnter")
        self.RB1.configure(state="normal")
        self.RB2.configure(state="normal")
        self.RB3.configure(state="normal")
        self.RB1.select()
        # invoke
        
    ###Create Second Window    
    def SecondWindow(self):
        
        child = tkinter.Toplevel(self)
        child.title("Comments")
        ###Create the main container frames
        windowfirst = Frame(child, bg='yellowgreen', width = 1000, height=50, padx=3, pady=3)
        windowsecond = Frame(child, bg='gray', width=1000, height=100, padx=3, pady=3)

        ###layout all of the main containers
        windowfirst.pack()
        windowsecond.pack()

        ##comment 
        ctr_left = Frame(windowsecond, bg='red', width=500, height=300, padx=3, pady=3)
        ctr_right = Frame(windowsecond, bg='brown', width=500, height=300, padx=3, pady=3)

        ctr_left.grid(row=0, column = 0, sticky="ns")
        ctr_right.grid(row=0, column = 2, sticky="ns")

In [24]:
root = App(None)
root.title('Shopify Reddit Analyzer')
root.mainloop()

0
1
2
3
4
5
6
7
8
9
10
11
OnButtonClick


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/vibhorsachdeva/anaconda/lib/python3.5/site-packages/pandas/indexes/base.py", line 1986, in get_value
    return tslib.get_value_box(s, key)
  File "pandas/tslib.pyx", line 777, in pandas.tslib.get_value_box (pandas/tslib.c:17017)
  File "pandas/tslib.pyx", line 786, in pandas.tslib.get_value_box (pandas/tslib.c:16691)
TypeError: 'str' object cannot be interpreted as an integer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/vibhorsachdeva/anaconda/lib/python3.5/tkinter/__init__.py", line 1550, in __call__
    return self.func(*args)
  File "<ipython-input-23-32781f7a02f8>", line 116, in OnButtonClick
    searchTable = shopifySearch(df,self.entryVariable.get())
  File "<ipython-input-18-298bb3ef7204>", line 62, in shopifySearch
    writer.add_document(comment_ID = data['name'],
  File "/Users/vibhorsachdeva/anaconda/lib/python3.5/site-pa